# Generate Data 

## Pandas dataframes 

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timezone
from sklearn.datasets import make_hastie_10_2
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

def generate_entities(size):
    return np.random.choice(size, size=size, replace=False)

def generate_data(entities, year=2021, month=10, day=1) -> pd.DataFrame:
    n_samples=len(entities)
    X, y = make_hastie_10_2(n_samples=n_samples, random_state=0)
    df = pd.DataFrame(X, columns=["f0", "f1", "f2", "f3", "f4", "f5", "f6", "f7", "f8", "f9"])
    df["y"]=y
    df['entity_id'] = entities
    df['datetime'] = pd.to_datetime(
            np.random.randint(
                datetime(year, month, day, 0,tzinfo=timezone.utc).timestamp(),
                datetime(year, month, day, 22,tzinfo=timezone.utc).timestamp(),
                size=n_samples),
        unit="s", #utc=True
    )
    df['created'] = pd.to_datetime(
            datetime.now(), #utc=True
            )
    df['month_year'] = pd.to_datetime(datetime(year, month, day, 0, tzinfo=timezone.utc), utc=True)
    return df

entities=generate_entities(1000000)

entity_df = pd.DataFrame(data=entities, columns=['entity_id'])
entity_df["event_timestamp"]=datetime(2021, 1, 14, 23, 59, 42, tzinfo=timezone.utc)

## Create Delta Lake 

In [6]:
import time
for d in range(1,15):
    break # TMP :)
    print(f"DAY {d}")
    
    start_time = time.time()
    data=generate_data(entities,month=1, day=d)
    print(f"## GENERATED - {time.time() - start_time} s")
    
    start_time = time.time()
    spark.createDataFrame(data).write.format("delta").mode("append").partitionBy('month_year').save("./dataset/all")
    print(f"## DELTA CREATED - {time.time() - start_time} s")

DAY 1
## GENERATED - 1.9863653182983398 s
## DELTA CREATED - 118.46784734725952 s
DAY 2
## GENERATED - 2.2533488273620605 s
## DELTA CREATED - 113.56314516067505 s
DAY 3
## GENERATED - 2.090444326400757 s
## DELTA CREATED - 117.54949474334717 s
DAY 4
## GENERATED - 2.137775421142578 s
## DELTA CREATED - 113.69700503349304 s
DAY 5
## GENERATED - 2.0107674598693848 s
## DELTA CREATED - 112.49230170249939 s
DAY 6
## GENERATED - 2.04490327835083 s
## DELTA CREATED - 116.83132553100586 s
DAY 7
## GENERATED - 2.12314772605896 s
## DELTA CREATED - 114.3579614162445 s
DAY 8
## GENERATED - 2.1742141246795654 s
## DELTA CREATED - 115.68657755851746 s
DAY 9
## GENERATED - 2.001004695892334 s
## DELTA CREATED - 112.91505312919617 s
DAY 10
## GENERATED - 2.1537675857543945 s
## DELTA CREATED - 113.79394125938416 s
DAY 11
## GENERATED - 2.077458620071411 s
## DELTA CREATED - 116.54374861717224 s
DAY 12
## GENERATED - 2.2862818241119385 s
## DELTA CREATED - 119.25584959983826 s
DAY 13
## GENERATED - 

## Delta Lake history 

In [2]:
from delta.tables import *

deltaTable = DeltaTable.forPath(spark, "./dataset/all")

fullHistoryDF = deltaTable.history()
fullHistoryDF.show()

+-------+--------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|version|           timestamp|userId|userName|operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+--------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|     13|2022-02-11 01:08:...|  null|    null|    WRITE|{mode -> Append, ...|null|    null|     null|         12|  Serializable|         true|{numFiles -> 12, ...|        null|Apache-Spark/3.2....|
|     12|2022-02-11 01:06:...|  null|    null|    WRITE|{mode -> Append, ...|null|    null|     null|         11|  Serializable|         true|{numFiles -> 12, ...|        null|Apache-Spark/3.2....|
|     11|2

# Feast Apply 

In [2]:
!rm -r .ipynb_checkpoints
from feast.repo_operations import apply_total
from feast.repo_config import load_repo_config
from pathlib import Path

repo = Path('/home/jovyan/feast-pyspark/feature_repo/')

repo_config = load_repo_config(repo)
apply_total(repo_config, repo, True)

Created entity entity_id
Created feature view my_statistics

Created sqlite table repo_my_statistics



/opt/conda/lib/python3.9/site-packages/feast/feature_view.py:100: DeprecationWarning: The argument 'input' is being deprecated. Please use 'batch_source' instead. Feast 0.13 and onwards will not support the argument 'input'.
  warnings.warn(


In [3]:
import pyspark
edf = entity_df[entity_df.entity_id<=500]

edf = spark.createDataFrame(edf)

In [8]:
from feast import FeatureStore
import pandas as pd

import time
from feast_pyspark import SparkOfflineStore

store = FeatureStore(repo_path=".")

start_time = time.time()
training_df = store.get_historical_features(
    entity_df=edf, 
    features = [
        'my_statistics:f0',
        'my_statistics:f1',
        'my_statistics:f2',
        'my_statistics:f3',
        'my_statistics:f4',
        'my_statistics:f5',
        'my_statistics:f6',
        'my_statistics:f7',
        'my_statistics:f8',
        'my_statistics:f9',
        'my_statistics:y',
    ],
).to_df()


print("--- %s seconds ---" % (time.time() - start_time))

training_df

--- 0.27462100982666016 seconds ---


,entity_id,event_timestamp,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,y
0,474,2021-01-14 23:59:42,0.343930,-0.903572,-0.334380,0.579604,-0.858730,1.191023,0.038458,-1.355268,2.193533,0.368305,1.0
1,26,2021-01-14 23:59:42,1.509172,0.269034,0.076565,0.489345,0.029237,0.451975,-0.154569,-0.398226,-0.617431,0.522517,-1.0
2,29,2021-01-14 23:59:42,-1.218221,1.106411,1.140555,0.952641,-0.352052,0.950669,-2.090229,0.153703,0.244905,-1.630285,1.0
3,191,2021-01-14 23:59:42,0.941750,-1.058797,1.215369,0.491941,0.957801,1.250372,-1.197075,0.108491,0.540480,-2.494599,1.0
4,65,2021-01-14 23:59:42,0.114411,0.460069,0.088460,0.130579,-0.066583,-0.470531,-0.772649,-0.622088,-0.192670,-0.012996,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,340,2021-01-14 23:59:42,0.907920,1.486164,0.222268,0.448620,1.630479,-0.855669,-0.922604,-0.445574,-1.234989,-0.194179,-1.0
497,24,2021-01-14 23:59:42,-0.819402,-0.412769,2.430399,-0.704433,0.396821,1.805116,-0.580702,-0.227816,-0.549949,1.513524,1.0
498,458,2021-01-14 23:59:42,0.182731,0.553639,-2.673206,0.229878,-0.320017,0.317976,1.321337,-0.855418,1.317480,0.504258,1.0
499,469,2021-01-14 23:59:42,-0.884719,1.695464,0.127202,0.910863,-1.478744,0.713934,0.759517,-0.535436,-0.098829,-0.391195,-1.0


In [ ]:
from feast import utils
from datetime import datetime

from feast import FeatureStore
from feast_pyspark import SparkOfflineStore

start_date=utils.make_tzaware(datetime.fromisoformat('2020-01-03T14:30:00'))
end_date=utils.make_tzaware(datetime.fromisoformat('2023-01-03T14:30:00'))

store = FeatureStore(repo_path=".")

store.materialize(start_date=start_date,end_date=end_date)

In [ ]:
from pprint import pprint
from feast import FeatureStore

store = FeatureStore(repo_path=".")

feature_vector = store.get_online_features(
    features=[
        "my_statistics:f1",
        "my_statistics:f2",
        "my_statistics:f3",
    ],
    entity_rows=[
        {"entity_id": 1004},
        {"entity_id": 1005},
    ],
).to_df()

feature_vector